## Problem 2
[Link to LLM conversation](COPYPASTLINKHERE)

(Practice on simulating time inhomogenous Poisson processes.) Suppose that the customer arrivals from 11AM to 1PM follow a time inhomogeneous Poisson process. The arrival rate is time-varying and is linearly increasing between 11AM and 1PM. The arrival rate is 0.5 per minute at 11AM and the arrival rate is 1 per minute at 1PM. Suppose that there is one server in the system, serving customers in a first come first serve criterion. The service time requirement distribution for any customer is exponentially distributed with expectation as 35 seconds. Use 100 simulation replications for this Deliverable and provide confidence intervals with 95% of confidence level.

a) Use simulation to compute the expectation of the number of customers at time 1PM.Suppose that you are interested in knowing that on a typical Wednesday, what is the expectation of the averaged waiting time for customers who arrive at the food truck between 11:15 AM to 11:30 AM. Compute an estimate of this quantity using simulation.

In [ ]:
...

b) Use simulation to compute the expectation of averaged waiting time for all those cus-
tomers that arrive between 12:45 PM to 1 PM. 

*Note: The average waiting time is defined as for a given day, the total waiting times of all customers on that day divided by the number of customers on that day. Such average waiting time may vary from day to day and is a random variable itself. The expectation of it can be computed and approximated by taking multiple days and do a sample mean.*</li>

c.) Use simulation to compute the expectation of averaged waiting time for all those customers that arrive between 11:45 AM to 12:00 PM.

In [1]:
...

Ellipsis